In [145]:
%pip install --upgrade plotly

Note: you may need to restart the kernel to use updated packages.


In [146]:
# install pyspark
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [147]:
# import pyspark
import pyspark

In [148]:
# get access to spark
spark

In [149]:
# Convert Excel files to csvs
import pandas as pd


In [150]:
# Convert Excel files to csvs
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep=',', index=False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')


/Users/tharushavihanga/Developer/test_EDA/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default

/Users/tharushavihanga/Developer/test_EDA/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [151]:
# Read in Pandas dataframes
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entries_gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

In [152]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [153]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [154]:
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [155]:
import plotly.express as px

In [156]:
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], title='Total Medals by Country',color_discrete_sequence=['Gold', 'Silver', 'Brown'])
fig.show()

In [157]:
coaches.head(10)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN
5,AIKMAN Siegfried Gottlieb,Japan,Hockey,Men
6,AL SAADI Kais,Germany,Hockey,Men
7,ALAMEDA Lonni,Canada,Baseball/Softball,Softball
8,ALEKNO Vladimir,Islamic Republic of Iran,Volleyball,Men
9,ALEKSEEV Alexey,ROC,Handball,Women


In [158]:
spark.read.csv('Coaches.csv',header=True,inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [159]:
# Run the Spark SQL query and collect results
results = spark.sql('''
  SELECT NOC, COUNT(NOC) AS count 
  FROM Coaches 
  GROUP BY NOC
  ORDER BY NOC
''').collect()

# Convert to Pandas DataFrame
pd_coaches_query = pd.DataFrame(
  [(row['NOC'], row['count']) for row in results],
  columns=['NOC', 'count']
)

pd_coaches_query.head()

,NOC,count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1


In [160]:
fig  = px.bar(pd_coaches_query,x='NOC',y='count',title='Number of Coaches by Country',color='count')
fig.show()

In [161]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [162]:
spark.read.csv('Athletes.csv',header=True,inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('Select * from athletes order by NOC,Discipline').show(10)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
+--------------------+-----------+-------------------+
only showing top 10 rows



In [163]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline, 
                                 count(*) Count from athletes group by NOC, Discipline order 
                                 by NOC, Discipline''').show()

+-----------+-------------------+-----+
|        NOC|         Discipline|Count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [164]:
results = spark.sql(''' select first(NOC) as NOC, first(Discipline) as Discipline, 
                        count(*) Count from athletes group by NOC, Discipline order 
                         by NOC, Discipline''').collect()

# Convert to Pandas DataFrame
pd_athletes_query = pd.DataFrame(
  [(row['NOC'], row['Discipline'], row['Count']) for row in results],
  columns=['NOC', 'Discipline', 'Count']
)

pd_athletes_query.head(10)

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
1,Afghanistan,Shooting,1
2,Afghanistan,Swimming,1
3,Afghanistan,Taekwondo,1
4,Albania,Artistic Gymnastics,1
5,Albania,Athletics,1
6,Albania,Judo,1
7,Albania,Shooting,1
8,Albania,Swimming,2
9,Albania,Weightlifting,2


In [165]:
fig = px.bar(pd_athletes_query,x='NOC',y='Count',color='Discipline',title='Number of Athletes by Country and Discipline')
fig.show()

In [166]:
athletes['Discipline'].value_counts().head(10)

Discipline
Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: count, dtype: int64

In [167]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query.Discipline.value_counts()

Discipline
Athletics    195
Swimming     181
Football      24
Name: count, dtype: int64

In [168]:
fig = px.bar(pd_athletes_query,x='NOC',y='Count',color='Discipline',title='Number of Athletes by Country and Discipline')
fig.show()

In [169]:
entries_gender.head(10)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041
5,Badminton,86,87,173
6,Baseball/Softball,90,144,234
7,Basketball,144,144,288
8,Beach Volleyball,48,48,96
9,Boxing,102,187,289


In [170]:
len(entries_gender)

46

In [171]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'}, {'type':'domain'}]]*23

fig =  make_subplots(rows=23, cols=2, specs=specs, subplot_titles=entries_gender['Discipline'])
sub_figs = []
for index,row in entries_gender.iterrows():
    sub_fig = go.Pie(labels=['Female','Male'],values=[row['Female'],row['Male']])
    sub_figs.append(sub_fig)
    
k = 0
for i in range(1,24):
    for j in range(1,3):
        fig.add_trace(sub_figs[k],row=i,col=j)
        k+=1

fig.update_layout(showlegend=False, height=7000, width=600, title_text="Distribution of Gender amongst each game")
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()

In [172]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women
